# ДЗ2 по предмету "Технологии решения задач машинного обучения", Инженерия Данных, МИСИС

In [1]:
import time
import torch # тут весь базовый функционал
import torch.nn # тут все, что нужно для построение нейросетей
import torch.nn.functional # тут все функции для работы с нейросетями
import torch.optim # тут все по оптимизаторам
import torch.optim.lr_scheduler # тут планировщики обучения
import torch.amp # тут функции для mix-precision обучения
import torch.cuda # тут все для взаимодействия с ГПУ
import torch.utils.data # тут все для работы с данными
import warnings
warnings.filterwarnings('ignore')

### Загрузка данных и разделение на обучающую и валидационную выборки. Создание Datasets и DataLoaders

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

dt = pd.read_csv('.local/utk_vecs_train.csv')
train_dt, val_dt = train_test_split(dt, random_state=32)

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, return_targets=True):
        self._x = dataframe.drop(['age', 'gender', 'race'], axis=1).values.astype('float32')
        self._y = dataframe[['age', 'gender', 'race']].values.astype('float32')
        self._return_targets = return_targets

    def __len__(self):
        return len(self._x)

    def __getitem__(self, idx):
        x = self._x[idx]

        if not self._return_targets:
            return x

        y = self._y[idx]
        return x, y


train_dataset, val_dataset = Dataset(train_dt), Dataset(val_dt)
train_dataloader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)
val_dataloader = torch.utils.data.DataLoader(
    dataset=val_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=0
)

### Описание модели

In [4]:
class FaceAttributesModel(torch.nn.Module):
    def __init__(self, in_features):
        super().__init__()
        
        self._fc1 = torch.nn.Linear(in_features, 256)
        self._fc2 = torch.nn.Linear(256, 128)
        
        self._fc_age = torch.nn.Linear(128, 1)
        self._fc_gender = torch.nn.Linear(128, 2)
        self._fc_race = torch.nn.Linear(128, 5)
        
        self._relu = torch.nn.ReLU()

    def forward(self, x):
        x = self._fc1(x)
        x = self._relu(x)
        x = self._fc2(x)
        x = self._relu(x)

        age = self._fc_age(x)
        gender = self._fc_gender(x)
        race = self._fc_race(x)
        
        return age, gender, race

### Обучение и валидация модели с теми же параметрами, что и на лекции (для определения базовых значений метрик качества)
Т.к. на лекции при разбиении выборки на обучающую и валидационную не было задано random_state, то получить те же базовые значения метрик качества будет проблематично. Поэтому мы рассчитаем их еще раз, используя те же параметры модели, что использовались на лекции, но уже на наших обучающей и валидационной выборке. 

In [47]:
model = FaceAttributesModel(in_features=512)
loss_f_age = torch.nn.MSELoss()
loss_f_gender = torch.nn.CrossEntropyLoss()
loss_f_race = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=10, gamma=0.1)

num_epochs =  25

for epoch in range(num_epochs):
    print('=========== TRAIN ===========')
    model.train()
    lr_scheduler.step()
    sum_loss_value = 0.0
    for batch in train_dataloader:
        optimizer.zero_grad()
        data, targets = batch
        age, gender_proba, race_proba = model(data)

        loss_value_age = loss_f_age(age, targets[:, [0]])

        loss_value_gender = loss_f_gender(gender_proba, targets[:, 1].long())

        loss_value_race = loss_f_race(race_proba, targets[:, 2].long())
        
        loss_value = loss_value_age * 1.0 + loss_value_gender * 100.0 + loss_value_race * 40.0
        loss_value.backward()
        optimizer.step()
        
        sum_loss_value += loss_value.item()
    print(f'loss: {sum_loss_value / len(train_dataloader)}')
        
    print('=========== VAL ===========')
    model.eval()
    metric_value_age, metric_value_gender, metric_value_race, n = 0.0, 0.0, 0.0, 0
    for batch in val_dataloader:
        data, targets = batch
        with torch.no_grad():
            age, gender_proba, race_proba  = model(data)
            metric_value_age += torch.nn.functional.l1_loss(age, targets[:, [0]], reduction='sum')
            metric_value_gender += (gender_proba.argmax(1) == targets[:, 1]).sum()
            metric_value_race += (race_proba.argmax(1) == targets[:, 2]).sum()
            n += len(data)
    print(f'AGE MAE ERROR: {metric_value_age / n}')
    print(f'GENDER ACCURACY ERROR: {metric_value_gender / n}')
    print(f'RACE ACCURACY ERROR: {metric_value_race / n}')

=========== TRAIN ===========
loss: 208.55249645182292
=========== VAL ===========
AGE MAE ERROR: 7.370297431945801
GENDER ACCURACY ERROR: 0.8849999904632568
RACE ACCURACY ERROR: 0.7211999893188477
=========== TRAIN ===========
loss: 156.88446643371583
=========== VAL ===========
AGE MAE ERROR: 6.982017993927002
GENDER ACCURACY ERROR: 0.8813999891281128
RACE ACCURACY ERROR: 0.7074000239372253
=========== TRAIN ===========
loss: 152.64213972778322
=========== VAL ===========
AGE MAE ERROR: 7.135563850402832
GENDER ACCURACY ERROR: 0.8942000269889832
RACE ACCURACY ERROR: 0.7152000069618225
=========== TRAIN ===========
loss: 150.75963356933593
=========== VAL ===========
AGE MAE ERROR: 7.284883499145508
GENDER ACCURACY ERROR: 0.8956000208854675
RACE ACCURACY ERROR: 0.7157999873161316
=========== TRAIN ===========
loss: 147.9844390645345
=========== VAL ===========
AGE MAE ERROR: 6.973548412322998
GENDER ACCURACY ERROR: 0.8637999892234802
RACE ACCURACY ERROR: 0.7039999961853027
===========

Таким образом, в качестве baseline берем следующие значения метрик качества: 
- Age: **6.333637237548828**  (MAE)
- Gender: **0.9056000113487244** (Accuracy)
- Race: **0.7879999876022339** (Accuracy)

_В ноутбуке с лекции метрики были следующими: Age - 6.254171848297119, Gender - 0.9121999740600586, Race - 0.782800018787384_


### Обучение и валидация модели с подбором параметров и возможностью остановки на любой эпохе

In [10]:
import time
model = FaceAttributesModel(in_features=512)
loss_f_age = torch.nn.MSELoss()
loss_f_gender = torch.nn.CrossEntropyLoss()
loss_f_race = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=6, gamma=0.1)

num_epochs =  30
epoch = 0
exit_flag = False
while not exit_flag:
    model.train()
    lr_scheduler.step()
    sum_loss_value = 0.0
    for batch in train_dataloader:
        optimizer.zero_grad()
        data, targets = batch
        age, gender_proba, race_proba = model(data)

        loss_value_age = loss_f_age(age, targets[:, [0]])

        loss_value_gender = loss_f_gender(gender_proba, targets[:, 1].long())

        loss_value_race = loss_f_race(race_proba, targets[:, 2].long())
        
        loss_value = loss_value_age * 1.3 + loss_value_gender * 100.0 + loss_value_race * 40.0
        loss_value.backward()
        optimizer.step()
        
        sum_loss_value += loss_value.item()
    print(f'========== EPOCH {epoch} ============')
    print('=========== TRAIN =================')
    print(f'loss: {sum_loss_value / len(train_dataloader)}')
        
    model.eval()
    metric_value_age, metric_value_gender, metric_value_race, n = 0.0, 0.0, 0.0, 0
    for batch in val_dataloader:
        data, targets = batch
        with torch.no_grad():
            age, gender_proba, race_proba  = model(data)
            metric_value_age += torch.nn.functional.l1_loss(age, targets[:, [0]], reduction='sum')
            metric_value_gender += (gender_proba.argmax(1) == targets[:, 1]).sum()
            metric_value_race += (race_proba.argmax(1) == targets[:, 2]).sum()
            n += len(data)
    print('=========== VAL ==================')
    print(f'AGE MAE ERROR: {metric_value_age / n}')
    print(f'GENDER ACCURACY ERROR: {metric_value_gender / n}')
    print(f'RACE ACCURACY ERROR: {metric_value_race / n}')
    
    time.sleep(0.3)
    epoch += 1
    exit_flag = (input('stop iterarions (Y if yes)') == 'Y')

========== EPOCH 0 ============
=========== TRAIN =================
loss: 213.18783084106445
=========== VAL ==================
AGE MAE ERROR: 6.809619426727295
GENDER ACCURACY ERROR: 0.8980000019073486
RACE ACCURACY ERROR: 0.782800018787384
========== EPOCH 1 ============
=========== TRAIN =================
loss: 159.86607458496093
=========== VAL ==================
AGE MAE ERROR: 6.490073204040527
GENDER ACCURACY ERROR: 0.9089999794960022
RACE ACCURACY ERROR: 0.7799999713897705
========== EPOCH 2 ============
=========== TRAIN =================
loss: 144.5618419576009
=========== VAL ==================
AGE MAE ERROR: 6.611946105957031
GENDER ACCURACY ERROR: 0.9114000201225281
RACE ACCURACY ERROR: 0.7868000268936157
========== EPOCH 3 ============
=========== TRAIN =================
loss: 132.02694673055012
=========== VAL ==================
AGE MAE ERROR: 6.18134880065918
GENDER ACCURACY ERROR: 0.9124000072479248
RACE ACCURACY ERROR: 0.7942000031471252
========== EPOCH 4 ============

Видим, что в результате модификации веса лосс-функции для возраста (на 1.3 с 1.0), скорости обучения (0.001 вместо 0.01), количества шагов, после которых скорость обучения снижается в 10 раз (с 10 до 6) и остановки после 7й эпохи получили немного более хорошие метрики, как по сравнению с полученными нами выше, так и по сравнению с теми, что были получены на лекции:
- Age: **5.996629238128662**  (MAE)
- Gender: **0.9179999828338623** (Accuracy)
- Race: **0.8064000010490417** (Accuracy)

### Прогнозирование значений на контрольной выборке и сохранение результатов

In [12]:
test_df = pd.read_csv('.local/utk_vecs_test_wo_target.csv')
test_df.shape

(3705, 512)

In [17]:
test_tensor = torch.tensor(test_df.values.astype('float32'))
with torch.no_grad():
    age, gender_proba, race_proba  = model(test_tensor)

age.size(), gender_proba.size(), race_proba.size()

(torch.Size([3705, 1]), torch.Size([3705, 2]), torch.Size([3705, 5]))

In [20]:
gender = gender_proba.argmax(1)
race = race_proba.argmax(1)
age.size(), gender.size(), race.size()

(torch.Size([3705, 1]), torch.Size([3705]), torch.Size([3705]))

In [43]:
dt = pd.DataFrame()
dt['age'] = age.numpy().flatten()
dt['gender'] = gender.numpy()
dt['race'] = race.numpy()

# Вот этот файлик вы мне и отправляете
dt.to_csv('submission_DL.csv', index=False)
dt

,age,gender,race
0,26.170326,0,3
1,67.896797,1,1
2,36.564541,0,3
3,22.143627,1,0
4,5.832328,0,0
...,...,...,...
3700,23.871128,0,2
3701,34.399456,0,0
3702,37.774616,1,1
3703,52.400341,0,0
